# Quickstart

In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [2]:
llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a tool that can be used to generate test cases for software systems. Here are some ways in which Langsmith can help with testing:\n\n1. Automated Testing: Langsmith can be used to automatically generate test cases based on the structure of the software system being tested. This can save time and effort compared to manual testing, where testers must write and execute tests manually.\n2. Coverage Analysis: Langsmith can analyze the coverage of the generated test cases and identify areas of the software that have not been thoroughly tested. This can help developers prioritize their testing efforts and ensure that all parts of the system are adequately covered.\n3. Test Case Generation: Langsmith can generate test cases based on the structure of the software system, including classes, methods, and variables. This can help ensure that all possible combinations of inputs and conditions are tested.\n4. Regression Testing: Langsmith can be used to generate test cases for regres

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm 

In [5]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that LangSmith can greatly assist with testing. Here are some ways in which LangSmith can aid in the testing process:\n\n1. Content Creation: With LangSmith's language generation capabilities, you can create high-quality content quickly and efficiently. This can help testers to generate a wide range of test cases, including edge cases and corner cases, which can be difficult to identify manually.\n2. Automated Testing: LangSmith can assist in automating testing by generating test cases and executing them automatically. This can save time and resources, as well as reduce the risk of human error.\n3. Test Data Generation: LangSmith can generate test data automatically, including input values, output values, and expected results. This can help testers to create comprehensive test plans and execute tests more quickly.\n4. Test Case Prioritization: LangSmith can prioritize test cases based on risk and 

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [7]:
chain = prompt | llm | output_parser

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Lingua Smith (Langsmith) is a valuable tool in the realm of language testing. Here are some ways Langsmith can assist with testing:\n\n1. **Multi-language support**: Langsmith's ability to handle multiple languages makes it an excellent tool for testing the accuracy and consistency of translations across different languages. By comparing the output of Langsmith with the original text, you can identify any discrepancies or inconsistencies in translation.\n2. **Customizable tests**: Langsmith allows you to create custom tests tailored to your specific needs. You can use these tests to evaluate the quality of translations, ensuring that they meet your standards and requirements.\n3. **Side-by-side comparison**: Langsmith's side-by-side comparison feature enables you to easily compare the original text with the translated version. This allows you to identify any differences or inconsistencies in the translation process.\n4

# Retrieval Chain

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [13]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results.'

In [14]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


According to the provided context, LangSmith can help with testing in several ways:

1. Prototyping: LangSmith allows users to rapidly experiment with different prompts, model types, and retrieval strategies, enabling them to quickly identify which combinations perform best.
2. Debugging: When developing new LLM applications, LangSmith provides native rendering of chat messages, functions, and retrieve documents, making it easier to understand how the model is performing and debug where it is failing.
3. Initial Test Set: LangSmith allows developers to create datasets of inputs and reference outputs, which can be used to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
4. Comparison View: When prototyping different versions of an application, LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the application.

# Conversation Retrieval Chain

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [17]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications,

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [19]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understan

# Agent

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [21]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(tavily_api_key="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")

In [22]:
tools = [retriever_tool, search]